# Introduction

This notebook is written as part of the hands-on tutorial for the LLNL CCMS Summer Institute Seminar 2022 (https://pls.llnl.gov/careers/internship-programs/computational-chemistry-and-materials-science-summer-institute) given by Prof Shyue Ping Ong.

If you are running this notebook from Google Colab, uncomment the next code box to install megnet first.

In [1]:
# !pip install megnet pymatgen==2022.7.24.1

In [1]:
from megnet.utils.models import load_model, AVAILABLE_MODELS
from pymatgen.core import Structure, Lattice
from mp_api.client import MPRester

2024-04-22 06:18:53.025859: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:18:53.025893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/shibuM/anaconda3/envs/megnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


By default, MEGNet already comes with some ready-to-use models trained on the Materials Project. 

In [2]:
print(AVAILABLE_MODELS)

['Eform_MP_2019', 'Eform_MP_2018', 'Efermi_MP_2019', 'Bandgap_classifier_MP_2018', 'Bandgap_MP_2018', 'logK_MP_2018', 'logG_MP_2018', 'logK_MP_2019', 'logG_MP_2019', 'QM9_Cv_2018', 'QM9_G_2018', 'QM9_H_2018', 'QM9_HOMO_2018', 'QM9_LUMO_2018', 'QM9_R2_2018', 'QM9_U_2018', 'QM9_U0_2018', 'QM9_ZPVE_2018', 'QM9_alpha_2018', 'QM9_gap_2018', 'QM9_mu_2018', 'QM9_omega1_2018']


We will first use the MPRester interface to get relaxed structures from the Materials Project (https://materialsproject.org). Here, we deliberately select two very different structures:
- bcc Mo, for which MP has already computed the elastic constants (https://materialsproject.org/materials/mp-129).
- Li10GeP2S12 (LGPS), a well-known lithium superionic conductor for which MP has not calculated the elastic constants (https://materialsproject.org/materials/mp-696128).

In [3]:
mpr = MPRester()

# Mo: mp-129
# Li10GeP2S12: mp-696128

structures = {}

structures["Mo"] = mpr.get_structure_by_material_id("mp-129")
structures["LGPS"] = mpr.get_structure_by_material_id("mp-696128")

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 8050.49it/s]


In [4]:
for model_name in AVAILABLE_MODELS:
    if model_name.endswith("_2019"):
        model = load_model(model_name)
        model.metadata
        for name, structure in structures.items():
            if model_name.startswith("log"):
                prediction = 10 ** model.predict_structure(structure).ravel()[0]
            else:
                prediction = model.predict_structure(structure).ravel()[0]
            prop_name = model_name.split("_")[0].replace("log", "")
            print(f'The predicted {prop_name} for {name} is {prediction:.3f} {model.metadata["unit"].replace("log", "")}.')

2024-04-22 06:19:44.479149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:19:44.479258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:19:44.479323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:19:44.479386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou

The predicted Eform for Mo is -0.003 eV/atom.
The predicted Eform for LGPS is -1.278 eV/atom.
The predicted Efermi for Mo is 8.401 eV.
The predicted Efermi for LGPS is 1.467 eV.
The predicted K for Mo is 268.447 10(GPa).
The predicted K for LGPS is 23.273 10(GPa).
The predicted G for Mo is 74.273 10(GPa).
The predicted G for LGPS is 9.737 10(GPa).


We may make the following observations:
- The formation energy of Mo is correctly predicted to be close to 0 (elemental reference state).
- The K of Mo is predicted to be 268 GPa, quite close to the MP value of 262 GPa. Note that this is not surprising as Mo is part of the training data for the MEGNet model to begin with. 
- In the paper "Elastic Properties of Alkali Superionic Conductor Electrolytes from First Principles Calculations" by Deng et al. [DOI: 10.1149/2.0061602jes](http://dx.doi.org/10.1149/2.0061602jes), the Materials Virtual Lab calculated the bulk modulus of LGPS to be 27.3 GPa and the shear modulus to be 7.9 GPa. This is in very good agreement with the predictions, even though it they are not in the original dataset.